## Task 2: Dataset Visualization with FiftyOne

In [1]:
!uv pip install fiftyone==1.11.0

Using Python 3.11.14 environment at: /home/vanessa/.local/micromamba/envs/handsoncv
Resolved 110 packages in 76ms                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)-------------------     0 B/10.77 MiB           
⠙ Preparing packages... (0/1)------------------- 16.00 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 32.00 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 48.00 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 60.44 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 76.44 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 92.44 KiB/10.77 MiB         
⠙ Preparing packages... (0/1)------------------- 108.44 KiB/10.77 MiB        
⠙ Preparing packages... (0/1)------------------- 124.44 KiB/10.77 MiB        
⠙ Preparing packages... (0/1)------------------- 140.44 KiB/10.

In [2]:
import random
import numpy as np
import fiftyone.utils.random as four
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

import os
# Set FiftyOne App to work in Remote SSH
os.environ["FIFTYONE_DISABLE_NOTEBOOK_APP"] = "1"

import fiftyone as fo
fo.app_config.enable_notebook_app = True

# Configuration
PATH = "~/Documents/repos/Applied-Hands-On-Computer-Vision/Assignment-2/data/" #"~/mnt/gdrive/path/to/your/assessment_folder" # Update this path to where the data is
MOUNTED_PATH = os.path.expanduser(PATH)
fiftyone_dataset_name = "cilp_assessment"

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

###  Requirement 2.1 (Grouped Dataset)

In [ ]:
# Cleanup & Initialization
if fiftyone_dataset_name in fo.list_datasets():
    fo.delete_dataset(fiftyone_dataset_name)
    
dataset = fo.Dataset(fiftyone_dataset_name)
dataset.add_group_field("group", default="rgb")

# Process Cubes & Spheres Modalities
classes = ["cubes", "spheres"]
all_pairs = []

for label in classes:
    class_dir = Path(MOUNTED_PATH) / label
    rgb_dir = class_dir / "rgb"
    lidar_dir = class_dir / "lidar"
    
    if not rgb_dir.exists():
        print(f"Skipping {label}: {rgb_dir} not found.")
        continue
    
    # Get matching file stems (filenames without extensions)
    rgb_files = {f.stem: f for f in rgb_dir.glob("*") if f.suffix.lower() in [".png", ".jpg"]}
    lidar_files = {f.stem: f for f in lidar_dir.glob("*") if f.suffix.lower() == ".npy"}
    
    matching_stems = sorted(set(rgb_files.keys()) & set(lidar_files.keys()))
    print(f"Found {len(matching_stems)} pairs for class: {label}")
    
    for stem in tqdm(matching_stems, desc=f"Processing {label}"):
        group = fo.Group()
        
        rgb_sample = fo.Sample(filepath=str(rgb_files[stem]), group=group.element("rgb"))
        rgb_sample["ground_truth"] = fo.Classification(label=label.rstrip('s')) # Normalize to 'cube'/'sphere'
        
        lidar_sample = fo.Sample(filepath=str(lidar_files[stem]), group=group.element("lidar"))
        lidar_sample["ground_truth"] = fo.Classification(label=label.rstrip('s'))

        all_pairs.append((rgb_sample, lidar_sample))
    
# dataset.add_samples(samples)

# Select a subset (10%) for exploration and training 
random.seed(SEED)
random.shuffle(all_pairs)
subset_size = int(len(all_pairs) * 0.10)
subset = all_pairs[:subset_size]

# Flatten and add to dataset
final_samples = []
for rgb_s, lidar_s in subset:
    final_samples.extend([rgb_s, lidar_s])

dataset.add_samples(final_samples)
dataset.persistent = True
print(f"\n✅ Created grouped dataset: {dataset.name}")
print(f"Group slices: {dataset.group_slices}")

# session = fo.launch_app(dataset)

# # Print the instructions in case the app does not launch in the notebook cell
# print(f"\nOpen the App.")
# print(f"1. Click the 'Views' icon (bookmark symbol) to see 'test_low_conf' and 'test_suspicious'.")
# print(f"2. URL: {session.url}")

Found 9999 pairs for class: cubes


Processing cubes: 100%|██████████| 9999/9999 [00:03<00:00, 2819.77it/s]


Found 7123 pairs for class: spheres


Processing spheres: 100%|██████████| 7123/7123 [00:02<00:00, 2690.51it/s]

   0% ||--------------|    1/3424 [23.3ms elapsed, 1.3m remaining, 43.0 samples/s] 

 100% |███████████████| 3424/3424 [1.7s elapsed, 0s remaining, 2.1K samples/s]         

✅ Created grouped dataset: cilp_assessment
Group slices: ['rgb', 'lidar']


### Requirement 2.2 (Dataset Statistics)

In [ ]:
print("="*50)
print(f"DATASET REPORT: {fiftyone_dataset_name}")
print("="*50)

# Overall Counts
print(f"Total samples (all slices): {len(dataset)}")
print(f"Total grouped pairs: {len(dataset.select_group_slices('rgb'))}")

# Modality Specific Statistics
for slice_name in dataset.group_slices:
    view = dataset.select_group_slices(slice_name)
    counts = view.count_values("ground_truth.label")
    
    print(f"\nModality: {slice_name.upper()}")
    print(f" - Count: {len(view)}")
    print(f" - Classes: {counts}")
    
    # Get metadata for one sample (Dimensions & Type)
    sample = view.first()
    try:
        with Image.open(sample.filepath) as img:
            print(f" - Dimensions: {img.size} (Width x Height)")
            print(f" - Data Type: {img.mode}")
    except:
        print(" - Metadata: Unable to read file (check mount)")

# Train/Validation Split (80/20)
four.random_split(dataset, {"train": 0.8, "val": 0.2}, seed=SEED)

train_count = dataset.match_tags("train").select_group_slices("rgb").count()
val_count = dataset.match_tags("val").select_group_slices("rgb").count()

print(f"\nSplit sizes (on pairs):")
print(f" - Train: {train_count}")
print(f" - Validation: {val_count}")

print(f"\nSample fields:")
for field_name, field in dataset.get_field_schema().items():
    print(f"  - {field_name}: {field}")
print("="*60)

# Class Distribution Visualization
rgb_counts = dataset.select_group_slices("rgb").count_values("ground_truth.label")
plt.figure(figsize=(7, 4))
plt.bar(rgb_counts.keys(), rgb_counts.values(), color=['#4287f5', '#f54242'])
plt.title(f"Class Distribution (10% Subset, Seed={SEED})")
plt.ylabel("Number of Samples")
plt.show()

DATASET REPORT: cilp_assessment
Total samples (all slices): 1712
Total grouped pairs: 1712

Modality: RGB
 - Count: 1712
 - Classes: {'sphere': 721, 'cube': 991}

Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds



### Requirement 2.3 (Visual Exploration)

In [ ]:
# Launch the App
session = fo.launch_app(dataset)

# Requirements 2.3 Documentation Helper
print("\n" + "="*60)
print("ACTION REQUIRED FOR REQUIREMENT 2.3:")
print("="*60)
print("1. If the cell below is blank, check the 'PORTS' tab in VS Code.")
print("2. Ensure port 5151 is forwarded to 127.0.0.1:5151.")
print(f"3. Open this link in your LOCAL browser: http://127.0.0.1:5151")
print("4. IN THE APP: Click the 'Group' icon in the top bar to see RGB and LiDAR side-by-side.")
print("="*60)


ACTION REQUIRED FOR REQUIREMENT 2.3:
1. If the cell below is blank, check the 'PORTS' tab in VS Code.
2. Ensure port 5151 is forwarded to localhost:5151.
3. Open this link in your LOCAL browser: http://localhost:5151
4. IN THE APP: Click the 'Group' icon in the top bar to see RGB and LiDAR side-by-side.
